In [ ]:
# # DOWNLOAD MINICONDA
# !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
# # INSTALL MINICONDA
# bash ~/miniconda.sh -b -u -p ~/miniconda3
# # REMOVE MINICONDA INSTALLER
# rm -rf ~/miniconda.sh

--2023-12-30 12:12:05--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141613749 (135M) [text/x-sh]
Saving to: ‘miniconda.sh’

miniconda.sh         39%[======>             ]  52.93M  16.3MB/s    eta 5s     ^C


In [ ]:
# Local pipeline
# To load a diffusion pipeline locally, use git-lfs to manually download the checkpoint (in this case, runwayml/stable-diffusion-v1-5) to your local disk. This creates a local folder, ./stable-diffusion-v1-5, on your disk:


### INSTALL GIT LFS
# git-lfs install

### DOWNLOAD MODEL
## REMEMBER TO DELETE .GIT WHEN DONE - HOGS STORAGE AND IS ONLY NEEDED FOR VERSION CONTROL
# git clone https://huggingface.co/runwayml/stable-diffusion-v1-5


In [1]:
### CREATE CONDA ENVIRONMENT
# !conda create -n diffusers_test python=3.11 -y
### EXPORT ENVIRONMENT
# !conda env export > diffusers_test.yml
### CREATE ENVIRONMENT FROM FILE
# !conda env create -f diffusers_test.yml

In [2]:
### DOWNLOAD REPO
# !git clone https://github.com/huggingface/diffusers
# !cd diffusers && git clone https://huggingface.co/stabilityai/sd-vae-ft-mse-original
### DOWNLOAD FILE FROM SS
# !wget https://raw.githubusercontent.com/ShivamShrirao/diffusers/main/examples/dreambooth/train_dreambooth.py -O ss_train_dreambooth.py

In [5]:
#### install dependencies

# !cd diffusers && pip install . && cd examples/dreambooth && pip install -r requirements.txt && pip install -r requirements_flax.txt

In [3]:
### SET ACCELERATE CONFIG

# !accelerate config default

In [2]:
##### RUNNING A MODEL #####

# from diffusers import DiffusionPipeline
# import torch
# # pro = """ultra realistic full body portrait, blue eyes,  hyper detail,
# #  cinematic lightingCanon EOS R3, nikon, f/1.4, ISO 200, 1/160s, 8K, RAW, unedited
# #  , symmetrical balance, in-frame, 8K"""
# pro = """A photo of dog in a bucket"""
# neg_pro = """painting, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, deformed, ugly,
#  blurry, bad anatomy, bad proportions, extra limbs, cloned face, skinny, glitchy,
# double torso, extra arms, extra hands, mangled fingers, missing lips, ugly face, distorted face, extra legs """
# repo_id = '/home/emizam/github/diffusers/realistic-vision-v51'
# stable_diffusion = DiffusionPipeline.from_pretrained(repo_id, use_safetensors=True)
# stable_diffusion.to("cuda")
# # stable_diffusion.enable_xformers_memory_efficient_attention()
# # stable_diffusion.enable_model_cpu_offload
# stable_diffusion(prompt=pro,
# negative_prompt=neg_pro,num_inference_steps=50).images[0]

In [6]:
#### CONCEPT LIST ####


# Configure a list of concepts to finetune on top of the normal StableDiffusion model
# For this example, we will only use 1 new concept - but you can add multiple concepts here and tweak '--max_training_steps' accordingly

# -instance_prompt - the prompt we would type to generate the image we are attempting to fine tune

# -class_prompt - denotes a prompt without the unique identifier/instance. This prompt is used for generating "class images" for prior preservation. For our example, this prompt is - "a photo of a person" versus a photo of a specific person.

# -instance_data_dir - the location where our training images are stored for finetuning

# -class_data_dir - sample images for the general class of prompt we are fine tuning - if there are no images here, samples will be generated. Otherwise, you can provie ~20 images of the general concept you want to generate (but not the actual instance images that we finetune on)


### https://github.com/ccrngd1/StableDiffusionExperimenting/blob/main/ShivamShriraoSD%2BDM.ipynb
# [
#     {
#         "instance_prompt": "photo of cc person",
#         "class_prompt": "photo of a person",
#         "instance_data_dir": "./content/data/cc",
#         "class_data_dir": "./content/data/person"
#     }
# ]

In [4]:
##### FINE TUNING A MODEL #####

# MODEL_NAME = "/home/emizam/github/diffusers/stable-diffusion-2-base"
# VAE_NAME= "/home/emizam/github/diffusers/sd-vae-ft-mse"
# INSTANCE_DIR="/home/emizam/github/diffusers/playground/dog"
# OUTPUT_DIR="./output"
# PRECISION = "fp16"

# !accelerate launch /home/emizam/github/diffusers_environment/ss_train_dreambooth.py\
#   --pretrained_model_name_or_path=$MODEL_NAME  \
#   --pretrained_vae_name_or_path=$VAE_NAME \
#   --instance_data_dir=$INSTANCE_DIR \
#   --output_dir=$OUTPUT_DIR \
#   --instance_prompt="a photo of sks dog" \
#   --resolution=512 \
#   --train_batch_size=1 \
#   --gradient_accumulation_steps=1 \
#   --learning_rate=5e-6 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --max_train_steps=10 \
# #   --save_sample_prompt="photo of cc person" \
#   --revision=$PRECISION \
#   --num_class_images=50 \
# #   --concepts_list="concepts_list_cc.json"    